# Data clean
data:
1. NYPD Complaint Data Historic
https://data.cityofnewyork.us/resource/qgea-i56i.csv
2. Local Area Unemployment Statistics
https://data.ny.gov/resource/5hyu-bdh8.csv
3. Shapefiles of NYC zip codes 
https://data.cityofnewyork.us/Business/Zip-Code-Boundaries/i8iw-xf4u/data?no_mobile=true .

- Downlowad the NYPD Complaint Data Historic and Local Area Unemployment Statistics first, using the given links


In [1]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs
from matplotlib.pylab import plt
import numpy as np
%matplotlib inline
import warnings
import geopandas as gpd
from urllib.request import urlopen 
import json
import urllib
warnings.filterwarnings('ignore')
from zipfile import ZipFile

# Local Area Unemployment Statistics

In [2]:
# Using ZipFile to read large csv file
#zip_file = ZipFile('Local_Area_Unemployment_Statistics__Beginning_1976.csv.zip')
#Unemployment_data = pd.read_csv(zip_file.open('Local_Area_Unemployment_Statistics__Beginning_1976.csv'))
Unemployment_data = pd.read_csv('Local_Area_Unemployment_Statistics__Beginning_1976.csv')

In [3]:
Unemployment_data.shape

(89078, 7)

In [4]:
Unemployment_data.head()

,Area,Year,Month,Labor Force,Employed,Unemployed,Unemployment Rate
0,New York State,2023,1,9594400,9155000,439400,4.6
1,New York State,2023,2,9664000,9232800,431200,4.5
2,New York State,2023,3,9687400,9295600,391900,4.0
3,New York State,2023,4,9652700,9296800,355900,3.7
4,New York State,2023,5,9720600,9349700,370900,3.8


## Data Cleaning Tasks
1. Data year from 2013 to 2022
2. area within New York City
The boroughs are the Bronx, Brooklyn, Manhattan, Queens, and Staten Island. 
Each borough is coextensive with a respective county of the State of New York: The Bronx is Bronx County, Brooklyn is Kings County, Manhattan is New York County, Queens is Queens County, and Staten Island is Richmond County.
3. Converting an Datetime feild

In [5]:
# Keep data only from year 2012 to 2022
sanityindex = ((Unemployment_data['Year'] >=2013) & (Unemployment_data['Year'] <=2022))
Unemployment_data = Unemployment_data.loc[sanityindex]
Unemployment_data.shape

(26520, 7)

In [6]:
# Unique Area 
Unemployment_data.Area.unique()

array(['New York State', 'Albany County', 'Allegany County',
       'Bronx County', 'Broome County', 'Cattaraugus County',
       'Cayuga County', 'Chautauqua County', 'Chemung County',
       'Chenango County', 'Clinton County', 'Columbia County',
       'Cortland County', 'Delaware County', 'Dutchess County',
       'Erie County', 'Essex County', 'Franklin County', 'Fulton County',
       'Genesee County', 'Greene County', 'Hamilton County',
       'Herkimer County', 'Jefferson County', 'Kings County',
       'Lewis County', 'Livingston County', 'Madison County',
       'Monroe County', 'Montgomery County', 'Nassau County',
       'New York County', 'Niagara County', 'Oneida County',
       'Onondaga County', 'Ontario County', 'Orange County',
       'Orleans County', 'Oswego County', 'Otsego County',
       'Putnam County', 'Queens County', 'Rensselaer County',
       'Richmond County', 'Rockland County', 'Saratoga County',
       'Schenectady County', 'Schoharie County', 'Schuyler 

In [7]:
# Bronx, Brooklyn, Manhattan, Queens, and Staten Island
NYC_counties = ['Bronx County','Kings County','New York County','Queens County','Richmond County']
Unemployment_data = Unemployment_data[Unemployment_data['Area'].isin(NYC_counties)]
Unemployment_data.shape

(600, 7)

In [8]:
# Combine Year and Month, convert to Datetime
Unemployment_data['YearMonth'] = pd.to_datetime(Unemployment_data['Year'].astype(str) + '-' +
                                           Unemployment_data['Month'].astype(str).str.zfill(2))
Unemployment_data['YearMonth'] = Unemployment_data['YearMonth'].dt.to_period('M')

In [9]:
Unemployment_data.reset_index(inplace = True, drop = True)

In [10]:
Unemployment_data.head()

,Area,Year,Month,Labor Force,Employed,Unemployed,Unemployment Rate,YearMonth
0,Bronx County,2022,1,612200,546100,66100,10.8,2022-01
1,Bronx County,2022,2,612400,551600,60800,9.9,2022-02
2,Bronx County,2022,3,607800,555000,52800,8.7,2022-03
3,Bronx County,2022,4,600700,552800,47900,8.0,2022-04
4,Bronx County,2022,5,599000,554700,44300,7.4,2022-05


In [11]:
# Save the cleaned data
Unemployment_data.to_csv("Unemployment_data_cleaned.csv",index=False)

## NYPD Complaint Data Historic

The data disctionary is here:https://data.cityofnewyork.us/api/views/qgea-i56i/files/b21ec89f-4d7b-494e-b2e9-f69ae7f4c228?download=true&filename=NYPD_Complaint_Incident_Level_Data_Footnotes.pdf

The focus of this notebook would be on following columns - 'CMPLNT_NUM','BORO_NM','CMPLNT_FR_DT','CMPLNT_FR_TM','OFNS_DESC','PARKS_NM','Latitude', 'Longitude'

'CMPLNT_NUM' is a unique id for each complaint, 
'BORO' is name of borough where complaint was reported,
'CMPLNT_FR_DT' and 'CMPLNT_FR_TM' are date and time of complaint respectively, 
'OFNS_DESC' is the type of offence reported, 
'PARKS_NM' is name of park where complaint recorded (if any) and 
'Latitude', 'Longitude' are location of complaint.

In [12]:
# NYPD Complaint Data Historic 
# Using ZipFile to read large csv file
#zip_file = ZipFile('NYPD_Complaint_Data_Historic_20231206.csv.zip')
#Complaint_data = pd.read_csv(zip_file.open('NYPD_Complaint_Data_Historic_20231206.csv'))
Complaint_data = pd.read_csv('NYPD_Complaint_Data_Historic_20231206.csv')

In [13]:
Complaint_data.shape

(8353049, 35)

In [14]:
Complaint_data.columns

Index(['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT',
       'CMPLNT_TO_TM', 'ADDR_PCT_CD', 'RPT_DT', 'KY_CD', 'OFNS_DESC', 'PD_CD',
       'PD_DESC', 'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'BORO_NM',
       'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'JURIS_DESC', 'JURISDICTION_CODE',
       'PARKS_NM', 'HADEVELOPT', 'HOUSING_PSA', 'X_COORD_CD', 'Y_COORD_CD',
       'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'TRANSIT_DISTRICT',
       'Latitude', 'Longitude', 'Lat_Lon', 'PATROL_BORO', 'STATION_NAME',
       'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX'],
      dtype='object')

## Select the interested columns

In [15]:
selectedCol = ['CMPLNT_NUM','BORO_NM','CMPLNT_FR_DT','CMPLNT_FR_TM','LAW_CAT_CD','OFNS_DESC','PARKS_NM','Latitude', 'Longitude']
Complaint_data = Complaint_data[selectedCol]
Complaint_data.shape

(8353049, 9)

In [16]:
Complaint_data.head()

,CMPLNT_NUM,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,LAW_CAT_CD,OFNS_DESC,PARKS_NM,Latitude,Longitude
0,10600119,MANHATTAN,01/01/2002,11:00:00,FELONY,RAPE,(null),NaN,NaN
1,11052575,BRONX,04/22/2005,02:00:00,FELONY,GRAND LARCENY OF MOTOR VEHICLE,(null),NaN,NaN
2,10832306,BROOKLYN,07/11/2005,20:00:00,MISDEMEANOR,UNAUTHORIZED USE OF A VEHICLE,(null),NaN,NaN
3,10107192,BRONX,10/19/2005,20:30:00,FELONY,MISCELLANEOUS PENAL LAW,(null),NaN,NaN
4,23893731,BRONX,12/04/2005,09:00:00,FELONY,GRAND LARCENY,(null),NaN,NaN


## Data Cleaning task

### 1. Filter out missing/wrong date and times, missing borough name and duplicate complaints from the data

In [17]:
# Ensure date as datetime
Complaint_data['CMPLNT_FR_DT'] = pd.to_datetime(Complaint_data['CMPLNT_FR_DT'],errors = 'coerce')

In [18]:
# Check borough names
Complaint_data['BORO_NM'].unique()

array(['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND',
       '(null)'], dtype=object)

In [19]:
# 'CMPLNT_FR_DT' using NaT indicating missing data
Complaint_data[Complaint_data['CMPLNT_FR_DT'].isna() == True ].head()

,CMPLNT_NUM,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,LAW_CAT_CD,OFNS_DESC,PARKS_NM,Latitude,Longitude
397,146577202,QUEENS,NaT,12:00:00,MISDEMEANOR,SEX CRIMES,(null),40.769495,-73.915024
749,148451238,MANHATTAN,NaT,16:45:00,FELONY,FELONY ASSAULT,(null),40.765024,-73.984836
3012,148017249,MANHATTAN,NaT,12:11:00,FELONY,GRAND LARCENY,(null),40.800930,-73.941098
3183,148457236,QUEENS,NaT,12:30:00,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,(null),40.743125,-73.956068
5517,147212244,BRONX,NaT,16:00:00,FELONY,MISCELLANEOUS PENAL LAW,(null),40.816206,-73.896001


In [20]:
# 'CMPLNT_FR_TM' using '(null)' indicating missing data
Complaint_data[Complaint_data['CMPLNT_FR_TM'] ==  '(null)'].head()

,CMPLNT_NUM,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,LAW_CAT_CD,OFNS_DESC,PARKS_NM,Latitude,Longitude
722862,29289078,MANHATTAN,2007-05-07,(null),MISDEMEANOR,PETIT LARCENY,(null),40.814803,-73.956284
859572,33603135,MANHATTAN,2007-08-04,(null),MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF,(null),40.851213,-73.935216
863369,33622397,MANHATTAN,2007-08-05,(null),MISDEMEANOR,PETIT LARCENY,(null),40.765024,-73.984836
1001018,35482822,BROOKLYN,2007-11-03,(null),MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF,(null),40.664121,-73.947765
1037872,36133966,QUEENS,2007-12-02,(null),MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF,(null),40.743481,-73.874004


In [21]:
# 'BORO_NM' using '(null)' indicating missing data
Complaint_data[Complaint_data['BORO_NM'] ==  '(null)'].head()

,CMPLNT_NUM,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,LAW_CAT_CD,OFNS_DESC,PARKS_NM,Latitude,Longitude
606,32312820,(null),2005-08-01,08:00:00,FELONY,GRAND LARCENY,(null),40.602216,-74.002951
1546,36032787,(null),2002-04-09,12:00:00,FELONY,MISCELLANEOUS PENAL LAW,(null),40.840781,-73.936245
4222,46254129,(null),2003-09-01,00:01:00,FELONY,THEFT-FRAUD,(null),40.689464,-73.924029
5235,54015493,(null),2004-08-12,12:01:00,FELONY,FORGERY,(null),40.720442,-74.006744
5830,53988769,(null),1998-04-01,00:01:00,MISDEMEANOR,SEX CRIMES,(null),40.630600,-73.973705


In [22]:
# filter out missing/wrong date and times, missing borough name
sanityindex = ((Complaint_data['CMPLNT_FR_DT'].isna() == False) 
               & (Complaint_data['CMPLNT_FR_DT']!= '(null)')
               & (Complaint_data['CMPLNT_FR_TM'].isna() == False) 
               & (Complaint_data['CMPLNT_FR_TM']!= '(null)')
               & (Complaint_data['BORO_NM'] != '(null)'))
Complaint_data = Complaint_data.loc[sanityindex]
Complaint_data.shape

(8345417, 9)

In [23]:
# fitering duplicate complaint number 
Complaint_data = Complaint_data.drop_duplicates(subset = ['CMPLNT_NUM'])
Complaint_data.shape

(8345414, 9)

### 2. Remove rows where location is parks or greenspace, Keep rows for 2013 - 2022

In [24]:
# If a crime has occured outside a parkspace, this value would be NaN
sanityindex = (Complaint_data['PARKS_NM'] == '(null)')
Complaint_data = Complaint_data.loc[sanityindex]
Complaint_data.shape

(8311189, 9)

In [25]:
# Check the statring and ending date and time
dateStart = Complaint_data['CMPLNT_FR_DT'].min()
dateEnd = Complaint_data['CMPLNT_FR_DT'].max()
timeStart = Complaint_data[Complaint_data['CMPLNT_FR_DT']==dateStart]['CMPLNT_FR_TM'].min()
timeEnd = Complaint_data[Complaint_data['CMPLNT_FR_DT']==dateEnd]['CMPLNT_FR_TM'].max()
# Combine
dtStart = pd.to_datetime(str(dateStart)+' '+str(timeStart))
dtEnd = pd.to_datetime(str(dateEnd)+' '+str(timeEnd))

In [26]:
## check the timeline of data
(dtStart, dtEnd)

(Timestamp('1900-03-10 19:00:00'), Timestamp('2022-12-31 23:55:00'))

In [27]:
# Romove data before 2012
Complaint_data = Complaint_data[(Complaint_data['CMPLNT_FR_DT'] >= pd.to_datetime('01/01/2013'))]
Complaint_data.shape

(4680561, 9)

In [28]:
# Check the statring and ending date and time
dateStart = Complaint_data['CMPLNT_FR_DT'].min()
dateEnd = Complaint_data['CMPLNT_FR_DT'].max()
timeStart = Complaint_data[Complaint_data['CMPLNT_FR_DT']==dateStart]['CMPLNT_FR_TM'].min()
timeEnd = Complaint_data[Complaint_data['CMPLNT_FR_DT']==dateEnd]['CMPLNT_FR_TM'].max()
# Combine
dtStart = pd.to_datetime(str(dateStart)+' '+str(timeStart))
dtEnd = pd.to_datetime(str(dateEnd)+' '+str(timeEnd))

In [29]:
## check the timeline of cleaned data
(dtStart, dtEnd)

(Timestamp('2013-01-01 00:00:00'), Timestamp('2022-12-31 23:55:00'))

### 3. keep type 1 crimes as defined by FBI from the data :

The crime type is present in the 'OFNS_DESC' column. 
Keep the following categories: 'ARSON', 'BURGLARY', 'FELONY ASSAULT', 'GRAND LARCENY' ,'GRAND LARCENY OF MOTOR VEHICLE', 'MURDER & NON-NEGL. MANSLAUGHTER', 'RAPE', 'ROBBERY'

In [30]:
crimetypes = ['ARSON', 'BURGLARY', 'FELONY ASSAULT', 'GRAND LARCENY' ,'GRAND LARCENY OF MOTOR VEHICLE', 'MURDER & NON-NEGL. MANSLAUGHTER', 'RAPE', 'ROBBERY']
sanityindex = Complaint_data['OFNS_DESC'].isin(crimetypes) 
Complaint_data = Complaint_data.loc[sanityindex]
Complaint_data.shape

(1033729, 9)

In [31]:
Complaint_data.reset_index(inplace = True, drop = True)
Complaint_data

,CMPLNT_NUM,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,LAW_CAT_CD,OFNS_DESC,PARKS_NM,Latitude,Longitude
0,109325830,BRONX,2014-03-25,18:00:00,FELONY,RAPE,(null),40.887314,-73.847272
1,148859994,BRONX,2015-12-26,23:30:00,FELONY,RAPE,(null),40.837842,-73.919628
2,171216196,BRONX,2017-11-04,13:20:00,FELONY,ROBBERY,(null),40.837842,-73.919628
3,183871546,BROOKLYN,2018-06-15,05:04:00,FELONY,RAPE,(null),40.664121,-73.947765
4,206878965,BROOKLYN,2019-12-30,13:30:00,FELONY,ROBBERY,(null),40.671130,-73.913502
...,...,...,...,...,...,...,...,...,...
1033724,261180746,MANHATTAN,2022-12-31,22:00:00,FELONY,FELONY ASSAULT,(null),40.763545,-73.985192
1033725,261161441,MANHATTAN,2022-12-31,03:00:00,FELONY,GRAND LARCENY,(null),40.751442,-73.976049
1033726,261173446,BROOKLYN,2022-12-31,15:00:00,FELONY,ROBBERY,(null),40.698304,-73.953147
1033727,261152819,QUEENS,2022-12-30,09:00:00,FELONY,BURGLARY,(null),40.792606,-73.844008


### 4. Keep rows with location within NYC


In [32]:
## zip codes map
zips = gpd.read_file('ZIP_CODE_040114/ZIP_CODE_040114.shp')
zips.head()

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((1038098.252 188138.380, 1038141.936 ..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.713 186926.440, 1002314.243 ..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1011174.276 183696.338, 1011373.584 ..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995908.365 183617.613, 996522.848 18..."
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((991997.113 176307.496, 992042.798 17..."


In [33]:
# chekc Projection
zips.crs

<Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - New York - counties of Bronx; Kings; Nassau; New York; Queens; Richmond; Suffolk.
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [34]:
# make points from given latitudes and longitudes
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(Complaint_data.Longitude, Complaint_data.Latitude)]

geoComplaintLatLon = gpd.GeoDataFrame(Complaint_data,geometry=geometry,crs={'init': 'EPSG:4326'})
geoComplaintLatLon.shape

(1033729, 10)

In [35]:
# remove rows with location outside NYC

# spatial join 
ComplaintDatawithin = gpd.sjoin(geoComplaintLatLon,zips.to_crs({'init': 'epsg:4326'}), op='within', how='inner')
ComplaintDatawithin.shape

(1033813, 23)

In [36]:
# Drop duplicates 
ComplaintDatawithin = ComplaintDatawithin.drop_duplicates(subset='CMPLNT_NUM')
ComplaintDatawithin.shape

(1033650, 23)

In [37]:
ComplaintDatawithin.head()

,CMPLNT_NUM,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,LAW_CAT_CD,OFNS_DESC,PARKS_NM,Latitude,Longitude,geometry,...,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN
0,109325830,BRONX,2014-03-25,18:00:00,FELONY,RAPE,(null),40.887314,-73.847272,POINT (-73.84727 40.88731),...,Bronx,68942.0,5.526249e+07,NY,Bronx,36,005,http://www.usps.com/,0.0,0.0
10,239164165,BRONX,2022-01-07,22:00:00,FELONY,RAPE,(null),40.887314,-73.847272,POINT (-73.84727 40.88731),...,Bronx,68942.0,5.526249e+07,NY,Bronx,36,005,http://www.usps.com/,0.0,0.0
11,238976313,BRONX,2022-01-09,09:00:00,FELONY,RAPE,(null),40.887314,-73.847272,POINT (-73.84727 40.88731),...,Bronx,68942.0,5.526249e+07,NY,Bronx,36,005,http://www.usps.com/,0.0,0.0
36,91030599,BRONX,2013-01-01,12:00:00,FELONY,GRAND LARCENY,(null),40.887314,-73.847272,POINT (-73.84727 40.88731),...,Bronx,68942.0,5.526249e+07,NY,Bronx,36,005,http://www.usps.com/,0.0,0.0
252,88424270,BRONX,2013-01-04,12:10:00,FELONY,ROBBERY,(null),40.887314,-73.847272,POINT (-73.84727 40.88731),...,Bronx,68942.0,5.526249e+07,NY,Bronx,36,005,http://www.usps.com/,0.0,0.0


In [38]:
# Keep the selected interested columns only 
selectedCol = ['CMPLNT_NUM','BORO_NM','CMPLNT_FR_DT','CMPLNT_FR_TM','LAW_CAT_CD','OFNS_DESC','PARKS_NM','Latitude', 'Longitude']
Complaint_data = ComplaintDatawithin[selectedCol]
Complaint_data.shape

(1033650, 9)

In [39]:
Complaint_data.reset_index(inplace = True, drop = True)

In [40]:
Complaint_data.head()

,CMPLNT_NUM,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,LAW_CAT_CD,OFNS_DESC,PARKS_NM,Latitude,Longitude
0,109325830,BRONX,2014-03-25,18:00:00,FELONY,RAPE,(null),40.887314,-73.847272
1,239164165,BRONX,2022-01-07,22:00:00,FELONY,RAPE,(null),40.887314,-73.847272
2,238976313,BRONX,2022-01-09,09:00:00,FELONY,RAPE,(null),40.887314,-73.847272
3,91030599,BRONX,2013-01-01,12:00:00,FELONY,GRAND LARCENY,(null),40.887314,-73.847272
4,88424270,BRONX,2013-01-04,12:10:00,FELONY,ROBBERY,(null),40.887314,-73.847272


In [41]:
# Save the cleaned data
Complaint_data.to_csv("Complaint_data_cleaned.csv",index=False)